In [8]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import pandas as pd
from os import path
import glob
import warnings
from scrapers.scrapepool import ScrapePool
from scrapers.wos import WosScraper
import linecache
import csv

# disable warnings, because pandas throws warnings for mixed type columns all the time..
warnings.filterwarnings('ignore')

%autoreload

In [10]:
# scraper settings
args = {
    'phantomjs_exec': 'C:\\phantomjs.exe',
    'urls': {
        'author_search': "http://apps.webofknowledge.com/WOS_AuthorSearch.do?product=WOS&action=Search&search_mode=AuthorSearch&AU=",
        'save_to': 'data\\wos\\out_exp\\'
    }
}

In [ ]:
df = pd.read_csv("data/wos/experts_knut.csv", sep=",")#, quotechar="\"", encoding="ISO-8859-1")
searched = set(["_".join(".".join(path.basename(f).split(".")[:-1]).split("_")[:-1]) for f in glob.glob(path.join(args['urls']['save_to'], "*.*")) if path.getsize(f) > 4])
keywords = list(set(df['name'].tolist())-searched)
print(str(len(keywords)))
#SID=F1zvccKnI6GLoDk33zb

53


In [ ]:
s = ScrapePool(WosScraper, keywords, args, chunk_size=100)
results = s.run()

connection tested successfully!
something went wrong while looking for WADHWA M
found 0 addresses for WADHWA M: 
found 0 addresses for ROSE M: 
found 0 addresses for VARMA M: 
found 0 addresses for SCHMIDT M: 
found 0 addresses for PELLEGRINO R: 
found 0 addresses for WHITE PL: 
found 0 addresses for FERNANDEZ JA: 
found 0 addresses for BARNETT D: 
found 0 addresses for TATE JR: 
found 0 addresses for MICHOTTE C: 
found 0 addresses for COURTE S: 


In [1]:
merged = dict()
for f in glob.glob(path.join(args['urls']['save_to'], "*.*")):
    if path.getsize(f) > 4:
        name= "_".join(".".join(path.basename(f).split(".")[:-1]).split("_")[:-1])
        merged[name] = linecache.getline(f, 2).replace("\"","").replace(";",",").rstrip()

NameError: name 'glob' is not defined

In [ ]:
df2 = pd.DataFrame.from_dict(merged, orient='index')
df2.columns = ['email']
df2

In [ ]:
df = df.set_index('name')
df

In [ ]:
all_merged = pd.concat([df, df2], axis=1, join_axes=[df.index])
all_merged

In [ ]:
all_merged.to_csv(path.join(args['urls']['save_to'], 'merged.csv'), sep=";", index=True, quotechar='"', quoting=csv.QUOTE_ALL)